In [29]:
import csv
MAX_VERSES = 199
#annotation_list = ["Copy of ENGESV_Romans_segments - English1.tsv", "Copy of ENGESV_Romans_segments - English2.tsv", "Copy of ENGESV_Romans_segments - English3.tsv", "Copy of ENGESV_Romans_segments - chinese1.tsv", "Copy of ENGESV_Romans_segments - chinese2_run.tsv", "Copy of ENGESV_Romans_segments - chinese3_Lin.tsv"]
annotation_list = []
annotation_list.append("Copy of ENGESV_Romans_segments - Romanian_David.tsv")
annotation_list.append("Copy of ENGESV_Romans_segments - Korean_Kathleen.tsv")
annotation_list.append("Copy of ENGESV_Romans_segments - Dutch_Huyen.tsv")
annotation_list.append("Copy of ENGESV_Romans_segments - Vietnamese_Huyen.tsv")

In [30]:
# Transfer the contents of a tsv file to a list for easier access
def tsv_to_list(annotation_file, max_verses):
    annotation_list = [] # rows are verses, columns are title, text, labels
    annotation = open(annotation_file)
    read_tsv = csv.reader(annotation, delimiter="\t") 
    verse_count = 1
    for row in read_tsv:
        if verse_count == max_verses:
            break
        annotation_list.append(row)
        verse_count += 1
    return annotation_list

In [31]:
def get_labels_for(annotation):
    labels = []
    for row in annotation:
        if row[5] == '?' or row[5] == 'X': # don't use unsure
            labels.append(None)
        elif row[2].upper() == 'X' and (row[3].upper() != 'X' and row[4].upper() == 'X'): # don't use pos + neg mixed labels
            labels.append(None) 
        elif row[2].upper() == 'X' and (row[3].upper() != 'X' and row[4].upper() != 'X'): # only pos
            labels.append(1)
        elif row[2].upper() == 'X' and (row[3].upper() == 'X' and row[4].upper() != 'X'): # pos + neu
            labels.append(0.5)
        elif row[3].upper() == 'X' and (row[2].upper() != 'X' and row[4].upper() != 'X'): # only neu 
            labels.append(0)
        elif row[4].upper() == 'X' and (row[3].upper() != 'X' and row[2].upper() != 'X'): # only neg
            labels.append(-1)
        elif row[4].upper() == 'X' and (row[3].upper() == 'X' and row[2].upper() != 'X'): # neg + neu
            labels.append(-0.5)       
    return labels

In [4]:
'''
english1 = tsv_to_list(annotation_list[0], MAX_VERSES)
english2 = tsv_to_list(annotation_list[1], MAX_VERSES)
english3 = tsv_to_list(annotation_list[2], MAX_VERSES)
chinese1 = tsv_to_list(annotation_list[3], MAX_VERSES)
chinese2 = tsv_to_list(annotation_list[4], MAX_VERSES)
chinese3 = tsv_to_list(annotation_list[5], MAX_VERSES)

english1_labels = get_labels_for(english1)
english2_labels = get_labels_for(english2)
english3_labels = get_labels_for(english3)
chinese1_labels = get_labels_for(chinese1)
chinese2_labels = get_labels_for(chinese2)
chinese3_labels = get_labels_for(chinese3)

annotations = []
annotations.append(english1_labels) 
annotations.append(english2_labels)
annotations.append(english3_labels)
annotations.append(chinese1_labels)
annotations.append(chinese2_labels) 
annotations.append(chinese3_labels) 


for annotation in annotations:
    verse_count = 0
    while verse_count < len(annotations[0]):
        if annotation[verse_count] == None:
            for annot in annotations:
                del annot[verse_count]
        else:
            verse_count += 1
'''

'\nfor annotation in annotations:\n    verse_count = 0\n    while verse_count < len(annotations[0]):\n        if annotation[verse_count] == None:\n            for annot in annotations:\n                del annot[verse_count]\n        else:\n            verse_count += 1\n'

In [68]:
romanian = tsv_to_list(annotation_list[2], MAX_VERSES)
romanian_labels = get_labels_for(romanian)

In [69]:
print(len(romanian_labels))
import csv
avg_file = open("RomanianAverageLabels", 'w')
for verse_count in range(len(romanian_labels)):
    write = str(romanian_labels[verse_count])+"\n"
    avg_file.write(write)

198


In [56]:
def get_speech_labels_for(annotation_file):
    labels = []
    annotation_list = []
    annotation = open(annotation_file)
    read_tsv = csv.reader(annotation, delimiter="\t") 
    verse_count = 1
    for row in read_tsv:
        if verse_count == MAX_VERSES:
            break
        annotation_list.append(row)
        verse_count += 1
    
    for row in annotation_list:
        if '?' in row[6] or row[6] == '': # don't use unsure
            labels.append(None)
        elif row[6].upper() == 'P': # only pos
            labels.append(1)
        elif row[6].upper() == 'NU' or row[6].upper() == 'NE': # only neu 
            labels.append(0)
        elif row[6].upper() == 'N' or row[6].upper() == 'NG': # only neg
            labels.append(-1)
        else:
            if 'P' in row[6].upper() and ('NU' in row[6].upper() or 'NE' in row[6].upper()): # pos + neu
                labels.append(0.5)
            elif ('N' in row[6].upper() or 'NG' in row[6].upper()) and ('NU' in row[6].upper() or 'NE' in row[6].upper()): # neg + neu
                labels.append(-0.5)
            else: # pos + neg mixed
                labels.append(None)
    return labels

In [60]:
# Return the text sentiment for a single verse from a tsv file
# Numerical range (-3 is unsure, -2 is pos+neg)
def get_text_sentiment_for_verse(verse_num, tsv_file):
    annotations = open(tsv_file)
    read_tsv = csv.reader(annotations, delimiter="\t")
    row_count = 0
    for row in read_tsv:
        if row_count == verse_num:
            if row[5] == '?' or row[5] == 'X': # unsure
                return -3
            elif row[2].upper() == 'X' and (row[3].upper() != 'X' and row[4].upper() == 'X'): # pos + neg mixed labels
                return -2 
            elif row[2].upper() == 'X' and (row[3].upper() != 'X' and row[4].upper() != 'X'): # only pos
                return 1
            elif row[2].upper() == 'X' and (row[3].upper() == 'X' and row[4].upper() != 'X'): # pos + neu
                return 0.5
            elif row[3].upper() == 'X' and (row[2].upper() != 'X' and row[4].upper() != 'X'): # only neu 
                return 0
            elif row[4].upper() == 'X' and (row[3].upper() != 'X' and row[2].upper() != 'X'): # only neg
                return -1
            elif row[4].upper() == 'X' and (row[3].upper() == 'X' and row[2].upper() != 'X'): # neg + neu
                return -0.5  
            else:
                print("outlier row:", row_count)
                return -3
        row_count += 1
    annotations.close()

In [61]:
def get_new_speech_labels_for(annotation_file):
    labels = []
    annotation_list = []
    annotation = open(annotation_file)
    read_tsv = csv.reader(annotation, delimiter="\t") 
    row_count = 0
    for row in read_tsv:
        if row_count == MAX_VERSES:
            break
        if len(row) > 9:
            if row[9] == '?' or row[9].upper() == 'X': # don't use unsure
                labels.append(None)
                #print(">9, unsure", row_count)
                row_count += 1
                continue
            elif row[6] == '' and row[7] == '' and row[8] == '':
                sentiment = get_text_sentiment_for_verse(row_count, annotation_file)
                #print(">9, sentiment", sentiment, row_count)
                if sentiment == -2 or sentiment == -3: labels.append(None)
                else: labels.append(sentiment)
        if row[6].upper() == 'X' and (row[7].upper() != 'X' and row[8].upper() == 'X'): # pos + neg labels
            labels.append(None) 
            #print("unsure", row_count)
        elif row[6].upper() == 'X' and (row[7].upper() != 'X' and row[8].upper() != 'X'): # only pos
            labels.append(1) 
            #print("pos", row_count)
        elif row[6].upper() == 'X' and (row[7].upper() == 'X' and row[8].upper() != 'X'): # pos + neu
            labels.append(0.5)
            #print("0.5", row_count)
        elif row[7].upper() == 'X' and (row[6].upper() != 'X' and row[8].upper() != 'X'): # only neu 
            labels.append(0) 
            #print("neu", row_count)
        elif row[8].upper() == 'X' and (row[7].upper() != 'X' and row[6].upper() != 'X'): # only neg
            labels.append(-1) 
            #print("neg", row_count)
        elif row[8].upper() == 'X' and (row[7].upper() == 'X' and row[6].upper() != 'X'): # neg + neu
            labels.append(-0.5)
            #print("-0.5", row_count)
        elif row[6] == '' and row[7] == '' and row[8] == '' and len(row) <= 9:
            sentiment = get_text_sentiment_for_verse(row_count, annotation_file)
            #print("sentiment", sentiment, row_count)
            if sentiment == -2 or sentiment == -3: labels.append(None)
            else: labels.append(sentiment)
        else:
            if len(row) <= 9:
                print("outlier row:", row_count)
                #print("outlier", row_count)
                labels.append(None)
        row_count += 1
    annotation.close()
    
    print(len(labels))
    return labels

In [65]:
# use get_new_speech_labels_for method if the annotation was done with the new method of speech annotation
labels = get_new_speech_labels_for(annotation_list[3])
print(len(labels))
import csv
avg_file = open("RomanianAverageLabels", 'w')
for verse_count in range(len(labels)):
    write = str(labels[verse_count])+"\n"
    avg_file.write(write)

198
198


In [6]:
'''
english1_labels = get_speech_labels_for(annotation_list[0])
english2_labels = get_speech_labels_for(annotation_list[1])
english3_labels = get_speech_labels_for(annotation_list[2])
chinese1_labels = get_speech_labels_for(annotation_list[3])
chinese2_labels = get_speech_labels_for(annotation_list[4])
chinese3_labels = get_speech_labels_for(annotation_list[5])

speech_annotations = []
speech_annotations.append(english1_labels) 
speech_annotations.append(english2_labels) 
speech_annotations.append(english3_labels) 
speech_annotations.append(chinese1_labels)
speech_annotations.append(chinese2_labels)
speech_annotations.append(chinese3_labels) 


for annotation in speech_annotations:
    verse_count = 0
    while verse_count < len(speech_annotations[0]):
        if annotation[verse_count] == None:
            for annot in speech_annotations:
                del annot[verse_count]
        else:
            verse_count += 1
'''

'\nfor annotation in speech_annotations:\n    verse_count = 0\n    while verse_count < len(speech_annotations[0]):\n        if annotation[verse_count] == None:\n            for annot in speech_annotations:\n                del annot[verse_count]\n        else:\n            verse_count += 1\n'

In [7]:
#speech averages

label_total = 0
annotator_count = 0
avg_english_splabels = []
avg_chinese_splabels = []

english = [speech_annotations[0], speech_annotations[1], speech_annotations[2]]
chinese = [speech_annotations[3], speech_annotations[4], speech_annotations[5]]
for l in range(len(speech_annotations[0])):
    for i in range(len(english)):
        if english[i][l] != None:
            label_total += english[i][l]
            annotator_count += 1
    if annotator_count == 0:
        avg_english_splabels.append(None)
    else:
        avg_english_splabels.append(label_total / annotator_count)
    annotator_count = 0
    label_total = 0
print(avg_english_splabels, len(avg_english_splabels), '\n')

label_total = 0
annotator_count = 0
for l in range(len(speech_annotations[3])):
    for i in range(len(chinese)):
        if chinese[i][l] != None:
            label_total += chinese[i][l]
            annotator_count += 1
    if annotator_count == 0:
        avg_chinese_splabels.append(None)
    else:
        avg_chinese_splabels.append(label_total / annotator_count)
    label_total = 0
    annotator_count = 0
print(avg_chinese_splabels, len(avg_chinese_splabels))

[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 0.0, 0.6666666666666666, 0.6666666666666666, 1.0, -1.0, 0.75, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -0.8333333333333334, 0.3333333333333333, -1.0, -1.0, -1.0, 0.0, 0.6666666666666666, -1.0, -0.6666666666666666, 1.0, 0.3333333333333333, -0.6666666666666666, 1.0, 1.0, 1.0, 0.3333333333333333, -0.5, 1.0, 1.0, 1.0, -0.6666666666666666, -1.0, -1.0, -1.0, 0.0, 0.6666666666666666, -1.0, -0.16666666666666666, 1.0, 0.0, 1.0, -0.6666666666666666, 1.0, -0.3333333333333333, 0.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 0.0, 0.0, 0.5, 1.0, -0.3333333333333333, 1.0, 1.0, 0.6666666666666666, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.5, 1.0, 0.6666666666666666, 0.6666666666666666, 0.3333333333333333, 1.0, 1.0, 0.0, 0.0, 0.8333333333333334, 0.6666666666666666, 1.0, -1.0, -0.5, 0.6666666666666666, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 1.0, 1.0, 1.0, 1.0

In [8]:
#text averages

label_total = 0
avg_english_labels = []
avg_chinese_labels = []
annotator_count = 0

english = [annotations[0], annotations[1], annotations[2]]
chinese = [annotations[3], annotations[4], annotations[5]]
for l in range(len(annotations[0])):
    for i in range(len(english)):
        if english[i][l] != None:
            label_total += english[i][l]
            annotator_count += 1
    if annotator_count == 0:
        avg_english_labels.append(None)
    else:
        avg_english_labels.append(label_total / annotator_count)
    label_total = 0
    annotator_count = 0

label_total = 0
annotator_count = 0
for l in range(len(annotations[3])):
    for i in range(len(chinese)):
        if chinese[i][l] != None:
            label_total += chinese[i][l]
            annotator_count += 1
    if annotator_count == 0:
        avg_chinese_labels.append(None)
    else:
        avg_chinese_labels.append(label_total / annotator_count)
    label_total = 0
    annotator_count = 0

In [15]:
print(len(avg_english_labels))
import csv
english_avg_file = open("C:/Users/shery/JupyterProjects/ChineseAverageLabels", 'a')
#out_file = csv.writer(chinese_avg_file, delimiter="\t") 
for verse_count in range(len(avg_english_labels)):
    write = str(avg_english_labels[verse_count])+"\n"
    english_avg_file.write(write)

198
